In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\jclayton\dictionary\dictionarybuild\dictionarybuild\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jclayton\dictionary\dictionarybuild\dictionarybuild\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 408/408 [00:00<00:00, 14279.43 examples/s]


In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [5]:
trainer.train()

 36%|███▋      | 500/1377 [09:04<17:51,  1.22s/it]

{'loss': 0.5118, 'grad_norm': 0.7966640591621399, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [18:59<06:52,  1.09s/it]

{'loss': 0.2684, 'grad_norm': 0.09304222464561462, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [26:30<00:00,  1.16s/it]

{'train_runtime': 1590.6235, 'train_samples_per_second': 6.918, 'train_steps_per_second': 0.866, 'train_loss': 0.32947656358595595, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.32947656358595595, metrics={'train_runtime': 1590.6235, 'train_samples_per_second': 6.918, 'train_steps_per_second': 0.866, 'total_flos': 405114969714960.0, 'train_loss': 0.32947656358595595, 'epoch': 3.0})

In [6]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

100%|██████████| 51/51 [00:11<00:00,  4.32it/s]

(408, 2) (408,)


In [7]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [10]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8529411764705882, 'f1': 0.8983050847457628}

In [11]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\jclayton\dictionary\dictionarybuild\dictionarybuild\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer.train()

                                                  
 33%|███▎      | 459/1377 [08:29<15:09,  1.01it/s]

{'eval_loss': 0.4615370035171509, 'eval_accuracy': 0.7720588235294118, 'eval_f1': 0.8567026194144838, 'eval_runtime': 14.8989, 'eval_samples_per_second': 27.385, 'eval_steps_per_second': 3.423, 'epoch': 1.0}


 36%|███▋      | 500/1377 [09:13<17:13,  1.18s/it]  

{'loss': 0.57, 'grad_norm': 3.483377695083618, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


                                                  
 67%|██████▋   | 918/1377 [17:07<07:12,  1.06it/s]

...

 73%|███████▎  | 1000/1377 [18:36<06:30,  1.04s/it]

{'loss': 0.3705, 'grad_norm': 0.17242372035980225, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


                                                   
100%|██████████| 1377/1377 [26:04<00:00,  1.14s/it]

{'eval_loss': 0.6536084413528442, 'eval_accuracy': 0.8357843137254902, 'eval_f1': 0.8873949579831932, 'eval_runtime': 15.4899, 'eval_samples_per_second': 26.34, 'eval_steps_per_second': 3.292, 'epoch': 3.0}
{'train_runtime': 1564.7082, 'train_samples_per_second': 7.033, 'train_steps_per_second': 0.88, 'train_loss': 0.3976772089496244, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3976772089496244, metrics={'train_runtime': 1564.7082, 'train_samples_per_second': 7.033, 'train_steps_per_second': 0.88, 'total_flos': 405114969714960.0, 'train_loss': 0.3976772089496244, 'epoch': 3.0})